In [83]:
import pandas as pd
import numpy as np
import missingno as ms
import seaborn as sns
import matplotlib.pyplot as plt

In [84]:
data= pd.read_csv('../input/bigmart-sales-prediction/Train.csv')
data.head()

In [85]:
print('shape',data.shape)
print(data.isnull().sum())
ms.matrix(data)

In [86]:
data['Item_Weight'].mean()
data['Item_Weight'] = data['Item_Weight'].fillna(data['Item_Weight'].mean())


In [87]:

data['Outlet_Size'] = data['Outlet_Size'].fillna("others")
data['Outlet_Size'].unique()

In [88]:
#now we dont have any missing values in dataset
#print(data.isnull().sum())


print(data['Outlet_Size'].value_counts())


In [89]:
print(data['Item_Type'].value_counts())

In [90]:
print(data['Outlet_Type'].value_counts())

In [91]:
#data['Item_Identifier'].unique()
#data= data.drop('Item_Identifier', axis =1)

EDA

In [92]:
sns.distplot(data['Item_Weight'])

In [93]:
sns.countplot(data['Outlet_Size'])

In [94]:
sns.distplot(data['Item_Outlet_Sales'])

In [95]:
#converting to log
data['Item_Outlet_Sales'] = np.log(1+data['Item_Outlet_Sales'])

In [96]:
sns.distplot(data['Item_Outlet_Sales'])

In [97]:
sns.countplot(data["Outlet_Location_Type"])

In [98]:
#dealing with categorical data
cat_data = data.select_dtypes('object')
cat_data.head()
num_data = data.drop(cat_data,axis =1)
num_data.head()

In [99]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_data = cat_data.apply(le.fit_transform)

cat_data.head()

new_data = cat_data.join(num_data)
new_data.head()

In [100]:
x= new_data.iloc[:,:-1]
y= new_data.iloc[:,-1]
x.head()

In [101]:
x= x.drop(columns = ['Item_Identifier','Outlet_Establishment_Year','Outlet_Identifier'])

In [102]:
x["Item_Visibility"] = x["Item_Visibility"].replace([0] , [x["Item_Visibility"].mean()])

In [103]:
x.head()

In [104]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3,random_state =0)

In [105]:
y_train.shape

In [106]:
# running models 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

def train(model,x,y):
    model.fit(x,y)
    predict = model.predict(x)

    cross_val = cross_val_score(model,x,y, scoring ='neg_mean_squared_error', cv=5)
    cross_val = np.abs(np.mean(cross_val))
    
    print("cv score = ",cross_val)
    

In [107]:
from sklearn.linear_model import LinearRegression , Ridge,Lasso


model = LinearRegression(normalize =True)
model.fit(x_train,y_train)
predict = model.predict(x_test)
best_score = model.score(x_test,y_test)
print("model score",best_score)
sns.regplot(y_test,predict)
plt.show()

cross_val = cross_val_score(model,x_train,y_train,scoring='neg_mean_squared_error',cv=5)

print('cross val negative mean score', cross_val.mean())
coef= pd.Series(model.coef_ ,x_train.columns).sort_values()
coef.plot(kind ='bar', title = 'model_classification')
plt.show()

In [108]:
model = Ridge(normalize = True)
train(model,x_train,y_train)

model.fit(x_train,y_train)
print('model score =',model.score(x_test,y_test))

coef= pd.Series(model.coef_ ,x_train.columns).sort_values()
coef.plot(kind ='bar', title = 'model_classification')

In [109]:
model = Lasso()

model.fit(x_train,y_train)
print('model score =',model.score(x_test,y_test))

train(model,x_train,y_train)
coef= pd.Series(model.coef_ ,x_train.columns).sort_values()
coef.plot(kind ='bar', title = 'model_classification')

In [110]:
from sklearn.tree import DecisionTreeRegressor
model =DecisionTreeRegressor()

model.fit(x_train,y_train)
print('model score =',model.score(x_test,y_test))

train(model,x_train,y_train)
coef= pd.Series(model.feature_importances_,x_train.columns).sort_values()
coef.plot(kind ='bar', title = 'model_classification')

hyperparameter tuning

In [111]:
from sklearn.model_selection import RandomizedSearchCV

max_features =['auto','sqrt']
max_depth = [int(x) for x in np.linspace(1,30,6)]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10] 

grid_search = { 'max_features' : max_features , 'max_depth': max_depth ,'min_samples_split' : min_samples_split , 'min_samples_leaf' : min_samples_leaf }
print(grid_search)


In [112]:
#random forest regressor 
from sklearn.ensemble import RandomForestRegressor 
rf = RandomForestRegressor()

rscv = RandomizedSearchCV(estimator = rf , param_distributions = grid_search , scoring = 'neg_mean_squared_error', n_iter = 10, cv = 5,verbose = 2, random_state = 20)
rscv.fit(x_train,y_train)

In [113]:
print(rscv.best_params_)
print(rscv.best_score_)
predict = rscv.predict(x_test)
sns.distplot(y_test-predict)